\begin{align*}
\huge \text{Using turboSETI with Docker}
\end{align*}

This tutorial has been designed to accomplish two objectives:
1. To aid readers in the process of setting-up turbo_seti for use within Docker
2. To provide helpful pointers when using turbo_seti

< Add the bit about what turbo_seti is >


< Add the bit about what Docker is >

# I.    Installing Docker and Building turbo_seti

<div class="alert alert-block alert-warning">
<b>Disclaimer:</b> This will work for Debian 9 and for Google Cloud Platform (GCP) Virtual Machines (VM) because it's their Linux distribution of choice.
</div>

## 1. Install Docker

***
<div class="alert alert-block alert-info">
<b>Note:</b> This part of the tutorial is pulled from an existing tutorial.
</div>

The original tutorial can be found __[here](https://tomroth.com.au/gcp-docker/)__

***

Open a terminal and copy and paste the code below into your session.  

<div class="alert alert-block alert-warning">
<b>Reminder:</b> Note that the following code is geared towards debian users, as Google Compute Instances use Debian as their distro of choice!  OS-specific documentation is available elsewhere online for installing Docker and will not be covered in this tutorial.  But, what I can say is that if you instead use ubuntu, switch out "debian" for "ubuntu" anywhere you see "debian" in the commands below.
</div>

```
    sudo apt update
    sudo apt install --yes apt-transport-https ca-certificates curl gnupg2 software-properties-common
    curl -fsSL https://download.docker.com/linux/debian/gpg | sudo apt-key add -
```

Then add the repository

`sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/debian $(lsb_release -cs) stable"
`

`sudo apt update
sudo apt install --yes docker-ce
`

Docker should now be installed, which means you can now run Docker shell commands.

If we stop here, you'd have to use `sudo` as a prefix to every Docker command.  You can fix this by adding yourself (i.e., your user) to the Docker group by running the following code:

`sudo usermod -aG docker $USER`

For this change to take effect, you must log out and log back in to "refresh" your session.

`logout`

To test if Docker has been installed correctly, run the code below.

`docker run hello-world`

If that was successful, congrats!  Docker is properly installed!

## 2. Git Clone the turbo_seti repository

### Prerequisites

Ensure you have `git` installed.  If you don't, don't fret, because all you have to do is open a terminal session and run `sudo apt update` and subsequently run `sudo apt install git`.

In order to run `turbo_seti`, you need to *have* `turbo_seti`.  

So, to do that, let's open a (or continue using) terminal session and run

`git clone https://UCBerkeleySETI/turbo_seti`

which will 
1. Create a directory named `turbo_seti` on your machine
2. Clone the `turbo_seti` repository to the newly minted `turbo_seti` directory

We are now ready to install turbo_seti using Docker!  To do so, while still in the `turbo_seti` directory, run the Docker command below:

`docker build --tag turbo_seti .`

To verify that everything installed correctly, run the Docker command `docker image ls` and look for `turbo_seti`.  If `turbo_seti` is listed as one of the Docker images, then congratulations!  You have just built `turbo_seti` as a Docker image!

### You now wield the full power of `turbo_seti`!

# 3.    Working with Google Cloud Storage Buckets:  Installing `gcsfuse`

For those of you who are looking to use `turbo_seti` using data stored in a Google Cloud Storage Bucket (hereafter "bucket"), the best way to work with your data will be to install `gcsfuse`.  `gcsfuse` provides convenient way to mount buckets to a local directory, which means your data is mountable on both a virtual machine (such as google compute engines) and/or your local machine.  Using `gcsfuse`, data is accessible using shell commands, python scripts, etc. because it mounts the bucket to emulate a traditional file storage system.

# II.    Using turbo_seti

< Add bit about how Docker images, containers work >

We can now run a `turbo_seti` container using the `turbo_seti` image we built before.  The command to do this is straightforward, but it pays to bear in mind **how** we would like to be using `turbo_seti`.

One of the questions you should be asking yourself before running a container is "*how many files and directories do I need to access to properly run my scripts?*"  The reason being that when you run a container, you won't have any access to your files.

For example, here's a picture showing you how to make a Docker container named "articuno" that runs using the `turbo_seti` image.

<img src="docker-seti-1.png" alt="Basic Docker container creation" title="A Docker container named 'articuno'" />

Let's break down this command:
```
docker run --name articuno -it turbo_seti:latest bash
```
docker | run | --name articuno | -it turbo_seti:latest | bash 
:-:|:-:|:-:|:-:|:-: 
Tells the system to interpret the command using Docker|Tells Docker that we wish to create a new container | Name the container so that in the future you don't have to reactivate it using its ID (the sequence of letters and numbers after "root@" in the image above) | Specifies the image (turbo_seti) and image version (:latest) from which to obtain a container from.  Enables the use of an interactive terminal inside of the container ("-it")| Enables processes within the container to use as much memory and swap memory as needed

So as you can see, after running the command to create a container named "articuno", we are in the Docker container.  Let's exit out of this container by using the same way you exit out of a terminal session: by using `exit` or `ctrl+d` or whatever tickles your fancy.  You will see that we return back to our location we were when we initially called on Docker to create the container.  Hooray, we successfully created a docker container!

Now that's all find and dandy, but in order to use the container to actually run `turbo_seti`, you need some `h5` files to run it on.  Let's refer back to the "articuno" example.  Let's say I want to run `turbo_seti` on `h5` data of the Green Bank Telescope follow-up L-Band observations of TESS targets.  Additionally, let's say all of these `h5` files are located in `/home/raffytraas14/seti_tess/l_band/`.  Now, since I created the container in the home directory, I **should** be seeing `seti_tess` in the `ls` command.  But where is it?  It's nowhere to be found!  This is just an artifact of how Docker works, but, when you create a container, you can tell Docker to "mount" one your directories or files onto the container, giving the container access to those files.  That is, you can simply tell Docker to give your container access to a specific directory upon the container's creation.  This is called using a "bind mount".

Let's explore bind mounting with another example.

So, the objective is that I want to create a Docker container that is both capable of running `turbo_seti` **and** of accessing the folder `l_band` that contains my `h5` files.  The way to do this will be similar to how we made "articuno" earlier.  Let's create another Docker container to demonstrate.

## Side-Note: Upgrading to Python3.8.5 in a Google Cloud Instance

Download the `.tar.xz` file for Python 3.8.5 from the official Python website.  Then if you unpack using `tar xvz <.tar to unpack>` (if this doesn't work for you, run `sudo swupd bundle-add xz` then try again), you can `cd` into the extracted directory and if you read the `README.rst`, you'll find that the installation commands are as follows:

<pre>./configure
make
make test
sudo make install
</pre>
